# Visualize Today's Traffic Accidents in the City of Austin
##### Miguel Novo Villar
##### December, 2022 (More to be added)

## Modules

In [37]:
#Import data science modules
import pandas as pd
import numpy as np
import matplotlib as plt

#Scraping
from sodapy import Socrata

#Time
from datetime import datetime, timedelta

#Geospatial
import geopandas as gpd
import contextily as cx

## Retrieve and transform data

##### We need to get data from data.austintexas.gov and autenticate to load the entire dataset

In [38]:
MyAppToken = "MyAppTokenfrom_data.austintexas.gov"

client = Socrata("data.austintexas.gov", #website
                MyAppToken, #token from portal
                 username="mymail@gmail.com", #username
                 password="mypasswords") #password

#Retrieve from the desired repository
results = client.get("dx9v-zd7x", limit = 1000000000)

#Zips
Austin_zips = gpd.read_file("Z:\Clases\CLASE\Master Rochester\Fall_2022\InterviewPrep\ReadAustinData\Zip Code\geo_export_5b8f0432-d113-478a-b2a5-18fba0156b66.shp")[["geoid10", "geometry"]]
Austin_zips.rename(columns = {"geoid10": "ZIP"}, inplace = True)

#Convert to pandas DataFrame, timestamp and sort by date descending
results_df = pd.DataFrame.from_records(results)
results_df["published_date"] = pd.to_datetime(results_df.published_date)
results_df= results_df.sort_values(by="published_date", ascending= False)

print("All time accidents recorded: ", results_df.shape[0])

All time accidents recorded:  303155


# Transform to geopandas

Create geodataframe with accidents from today

In [39]:
today = results_df[(results_df['published_date'] > str(pd.to_datetime(datetime.utcnow() - timedelta(hours=24))))]

gdf = gpd.GeoDataFrame(today, 
                        geometry=gpd.points_from_xy(today.longitude, today.latitude),
                        crs ="EPSG:4326").drop(["location","latitude","longitude"],axis = 1)
print("Accidents recorded today: ", today.shape[0])

Accidents recorded today:  151


## Visualize

#### Accidents today

In [40]:
accidents = gdf.drop("published_date", axis =1)

accidents.explore(column='issue_reported', legend=True, marker_kwds=dict(radius=5))

#### Accidents today - ZIP code Level

In [61]:
# Lamda function for intersecction
f = lambda x:np.sum(accidents.intersects(x))
Austin_zips["accidents_in_area"] = Austin_zips['geometry'].apply(f)

#Display intersections
Austin_zips.explore(column='accidents_in_area', legend=True, cmap  = "plasma_r")